In [1]:
def get_stats(numbers):
  minimum = min(numbers)
  maximum = max(numbers)
  return minimum, maximum

In [2]:
lengths = [63, 73, 72, 60, 67, 66, 71, 61, 72, 70]

minimum, maximum = get_stats(lengths)

print(f'Min: {minimum}, Max: {maximum}')

Min: 60, Max: 73


# 演示unpacking 语句和返回多个值的函数

In [5]:
first, second = 1, 2
assert first == 1
assert second == 2

def my_function():
    return 1, 2

first, second = my_function()
assert first == 1
assert second == 2

用*返回没有被捕获到的值

In [6]:
def get_avg_ratio(numbers):
  average = sum(numbers) / len(numbers)
  scaled = [x / average for x in numbers]
  scaled.sort(reverse=True)
  return scaled

longest, *middle, shortest = get_avg_ratio(lengths)

print(f'Longest: {longest:>4.0%}')
print(f'Shortest: {shortest:>4.0%}')

Longest: 108%


## 在闭包中使用外围变量

In [4]:
def sort_priority(values, group):
    found = False
    def helper(x):
        nonlocal found # Must declare nonlocal to modify
        if x in group:
            found = True # Seems simple
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

In [5]:
values = [3, 5, 1, 2, 4]
group = [1, 2, 3]
expected = True
result = sort_priority(values, group)
assert result == expected, f"Expected {expected}, but got {result}"

## 用None 和docstring描述默认值会变的参数

### 只会执行一次

In [2]:
from time import sleep
from datetime import datetime

def log(message, when=datetime.now()):
    print(f'{when}: {message}')

In [4]:
log('Hi there!')
sleep(1)
log('Hi there again!')

2023-08-02 06:36:58.791218: Hi there!
2023-08-02 06:36:58.791218: Hi there again!


when设置成None docstring中对行为进行描述

In [5]:
def log(message, when=None):
    """Log a message with a timestamp.

    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print(f'{when}: {message}')

In [6]:
log('Hi,there!')
sleep(1)
log('Hi,there!')

2023-08-02 06:53:35.884004: Hi,there!
2023-08-02 06:53:36.887283: Hi,there!


### 参数传惨 指定位置 和 按位置传入

In [8]:
def safe_division(number, divisor, ignore_overflow, ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [12]:
result = safe_division(1.0, 10**500, True, False)
print(result)

0


* keyword-only-argument

In [14]:
def safe_division_c(number, divisor,*, ignore_overflow=False, ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [16]:
safe_division_c(1.0, 10**500, True, False)

TypeError: safe_division_c() takes 2 positional arguments but 4 were given

In [24]:
result = safe_division_c(1.0, 10**500, ignore_overflow=True)
print(result)

assert safe_division_c(number=2, divisor=5, ignore_overflow=True) == 0.4
assert safe_division_c(divisor=5,number=2, ignore_overflow=False) == 0.4

0


* positional-only argument

# 类与接口

In [2]:
class SimpleGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, score):
        self._grades[name].append(score)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

In [3]:
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
book.report_grade('Isaac Newton', 95)
book.report_grade('Isaac Newton', 85)
print(book.average_grade('Isaac Newton'))

90.0


In [7]:
from collections import defaultdict

class BySubjectGradeBook:
    
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append(grade)
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

In [8]:
book = BySubjectGradeBook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75)
book.report_grade('Albert Einstein', 'Math', 65)
book.report_grade('Albert Einstein', 'Gym', 90)
book.report_grade('Albert Einstein', 'Gym', 95)
book.report_grade('Albert Einstein', 'Gym', 85)
print(book.average_grade('Albert Einstein'))

82.0


In [9]:
grades = []
grades.append((90, 0.3))
grades.append((80, 0.2))
grades.append((70, 0.2))
grades.append((60, 0.1))
grades.append((50, 0.1))
grades.append((0, 0.1))
total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight
print(average_grade)

68.00000000000001


## nametuple

In [10]:
from collections import namedtuple

Grade = namedtuple('Grade', ('score', 'weight'))
print(Grade(81, 0.3))

Grade(score=81, weight=0.3)
